In [9]:
import pandas as pd
import numpy as np
import json
import ast
from ast import literal_eval
from datetime import datetime

df = pd.read_csv('movies_dataset.csv',sep=",")
#como el none nos estaba dando problemas lo eliminamos con esta funcion
def parse_dict_str(value):
    if pd.isna(value):
        return None
    else:
        return ast.literal_eval(value)
def parse_dict_str2(value):
    if pd.isna(value):
        return None
    else:
        genres = ast.literal_eval(value)
        return ','.join([g['name'] for g in genres])
def parse_dict_str3(value):
    if pd.isna(value):
        return None
    else:
        languages = ast.literal_eval(value)
        return ','.join([c['name'] for c in languages])     
def parse_dict_str4(value):
    if pd.isna(value):
        return None
    else:
        try:
            companies = ast.literal_eval(value)
            if isinstance(companies, list):
                return ','.join([c['name'] for c in companies])
            else:
                return 'Unknown'
        except (ValueError, TypeError):
            # If the input value is not a valid dictionary or list, split it on spaces and process each word separately
            words = value.split()
            return ','.join(words)
        
def extract_year(date_string):
    if pd.isna(date_string):
        return None
    try:
        date_object = datetime.strptime(date_string, '%Y-%m-%d')
        return date_object.year
    except ValueError: 
        return None
dfnormalizedprocountri = pd.json_normalize(df['production_countries']).fillna('')
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
df['revenue'] = df['revenue'].astype(float)
df['return'] = df['revenue'].div(df['budget'])
df['revenue'] = df['revenue'].astype(float)
df['return'] = df['revenue'].div(df['budget'])
df['year'] = df['release_date'].apply(extract_year)
df['belongs_to_collection'] = df['belongs_to_collection'].apply(parse_dict_str)
dfnormalizedbelongs = pd.json_normalize(df['belongs_to_collection']).fillna('')
#dfnormalizedproduction = pd.json_normalize(df['production_companies']).fillna('')
df['production_countries'] = df['production_countries'].apply(parse_dict_str4)
df = pd.concat([df, dfnormalizedbelongs['name']], axis=1)
df['genres'] = df['genres'].apply(parse_dict_str2)
df['spoken_languages'] = df['spoken_languages'].apply(parse_dict_str3)
#df['production_companies'] = df['production_companies'].apply(parse_dict_str)
df['production_companies'] = df['production_companies'].apply(parse_dict_str4)
#dfnormalizedgenres = pd.json_normalize(df['name']).fillna('')
#df = pd.concat([df, dfnormalizedgenres], axis=1)
#df['year'] = df['year'].replace([np.inf, -np.inf], ^np.nan)
df['vote_count'] = df['vote_count'].fillna(0).astype(int)
df['year'] = df['year'].fillna(0).astype(int)
df['return'].fillna(0)
df = df.drop(['belongs_to_collection','video','imdb_id','adult','original_title','poster_path','homepage'], axis=1)
#df[['adult', 'name', 'budget', 'revenue', 'genres', 'homepage', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count', 'release_date','year']]	
column = df.pop('name')
df.insert(0,'name', column)
info = input('ingrese el titulo de la pelicula')
vote_count = df.loc[df['title'] == info , 'vote_count'].values[0]
score = df.loc[df['title'] == info, 'popularity'].values[0]
year = df.loc[df['title'] == info, 'year'].values[0]
vote_average = df.loc[df['title'] == info, 'vote_average'].values[0]
if vote_count > 2000:
    print('La película', info, 'fue estrenada en el año.', year, 'La misma cuenta con un total de', vote_count,'valoraciones, con un promedio de', vote_average)
else:
    print('la pelicula que busca no cumple con la condicion de mas de 2000 votos por ende no se devolvera ningun valor') 
print('el año fue :', year, 'y la puntuacion fue', score)


C:\Users\carlo\AppData\Local\Temp\ipykernel_18820\1264301317.py:8: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('movies_dataset.csv',sep=",")


,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return,year,name
0,30000000.0,"Animation,Comedy,Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,English,Released,NaN,Toy Story,7.7,5415,12.451801,1995,Toy Story Collection
1,65000000.0,"Adventure,Fantasy,Family",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"TriStar Pictures,Teitler Film,Interscope Commu...",United States of America,1995-12-15,262797249.0,104.0,"English,Français",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413,4.043035,1995,
2,0.0,"Romance,Comedy",15602,en,A family wedding reignites the ancient feud be...,11.7129,"Warner Bros.,Lancaster Gate",United States of America,1995-12-22,0.0,101.0,English,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92,NaN,1995,Grumpy Old Men Collection
3,16000000.0,"Comedy,Drama,Romance",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,81452156.0,127.0,English,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34,5.090760,1995,
4,0.0,Comedy,11862,en,Just when George Banks has recovered from his ...,8.387519,"Sandollar Productions,Touchstone Pictures",United States of America,1995-02-10,76578911.0,106.0,English,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173,inf,1995,Father of the Bride Collection
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,0.0,"Drama,Family",439050,fa,Rising and falling between a man and woman.,0.072051,,Iran,NaN,0.0,90.0,فارسی,Released,Rising and falling between a man and woman,Subdue,4.0,1,NaN,0,
45462,0.0,Drama,111109,tl,An artist struggles to finish his work while a...,0.178241,Sine Olivia,Philippines,2011-11-17,0.0,360.0,,Released,NaN,Century of Birthing,9.0,3,NaN,2011,
45463,0.0,"Action,Drama,Thriller",67758,en,"When one of her hits goes wrong, a professiona...",0.903007,American World Pictures,United States of America,2003-08-01,0.0,90.0,English,Released,A deadly game of wits.,Betrayal,3.8,6,NaN,2003,
45464,0.0,,227506,en,"In a small town live two brothers, one a minis...",0.003503,Yermoliev,Russia,1917-10-21,0.0,87.0,,Released,NaN,Satan Triumphant,0.0,0,NaN,1917,


In [13]:
import pandas as pd
import numpy as np
import json
df = pd.read_csv('credits.csv', sep = ',')
def get_actor( nombre_actor ):
    if pd.isna(nombre_actor):
        return None
    else:
        actors = ast.literal_eval(nombre_actor)
        return ','.join([c['name'] for c in actors])    
df['name'] = df['name'].apply(get_actor)
df['name']


ParserError: Error tokenizing data. C error: Expected 1 fields in line 192, saw 66


In [2]:
import pandas as pd
import numpy as np
import json
import ast
from ast import literal_eval
from datetime import datetime

df = pd.read_csv('movies_dataset.csv',sep=",")

def cantidad_filmaciones_mes(date_string):
    if pd.isna(date_string):
        return None
    try:
        date_object = datetime.strptime(date_string, '%Y-%m-%d')
        return date_object.month
    except ValueError: 
        return None
def cantidad_filmaciones_day(date_string):
    if pd.isna(date_string):
        return None
    try:
        date_object = datetime.strptime(date_string, '%Y-%m-%d')
        return date_object.day
    except ValueError: 
        return None
df['day'] = df['release_date'].apply(cantidad_filmaciones_day)
df['day'] = df['day'].fillna(0).apply(int)
df['month'] = df['release_date'].apply(cantidad_filmaciones_mes)
df['month'] = df['month'].fillna(0).apply(int)
conteo_meses = df['month'].value_counts()
conteo_dias = df['day'].value_counts()
info = input('ingrese el titulo de la pelicula')

print('el año fue :', year, 'y la puntuacion fue', score)

    
#meses = input('Ingrese el nombre de un mes con la primera letra en mayuscula (Ejemplo : Enero): ')
#conteos = conteo_meses.rename(index={1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril', 5: 'Mayo', 6: 'Junio', 7: 'Julio', 8: 'Agosto', 9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'})
#valor = conteos.loc[meses]
#print(valor ,'cantidad de películas fueron estrenadas en el mes de', meses)



C:\Users\carlo\AppData\Local\Temp\ipykernel_18820\2344187054.py:8: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('movies_dataset.csv',sep=",")


KeyError: 'score'

In [11]:
import pandas as pd 
import numpy as np
import json 
import ast
df = pd.read_csv('movies_dataset.csv',sep=",")
df['belongs_to_collection']

C:\Users\carlo\AppData\Local\Temp\ipykernel_42952\3058238481.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('movies_dataset.csv',sep=",")


0        {'id': 10194, 'name': 'Toy Story Collection', ...
1                                                      NaN
2        {'id': 119050, 'name': 'Grumpy Old Men Collect...
3                                                      NaN
4        {'id': 96871, 'name': 'Father of the Bride Col...
                               ...                        
45461                                                  NaN
45462                                                  NaN
45463                                                  NaN
45464                                                  NaN
45465                                                  NaN
Name: belongs_to_collection, Length: 45466, dtype: object